# Chained MF for outfit recommendation
The workflow will be as follows
1. Convert all items into a scene.
2. Pass the scene into a MF factorization that is inverted to output an item
3. Request 3 items for each item

## Update the item to recommend

In [16]:
#test shirt = d4ad28d585788409ab2774b3780b6ce9
#
source_item = "d4ad28d585788409ab2774b3780b6ce9"

## Prerequisite
This is to create the dictionary that will convert a item into a scene

In [18]:
!pip install surprise

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [39]:
import csv
import copy

with open('fashion.csv', 'r') as f:
    
    csv_reader = csv.DictReader(f)
    list_of_items = list(csv_reader)

scene_list = {}

for item in list_of_items:
    productID = item.get('ProductID') 
    
    #if the product key exist
    if productID in scene_list:
        buffer = scene_list[productID]
        buffer.append(item.get('SceneID').strip())

    #if the product key is missing
    else:
        scene_list[productID] = [item.get('SceneID').strip()]

master_scene_list = copy.deepcopy(scene_list)


In [20]:
#sratch for testing delete afterwards!
print(scene_list['0027e30879ce3d87f82f699f148bff7e'])


['cdab9160072dd1800038227960ff6467', '14f59334af4539132981b1324a731067', 'e7d32df9f45b691afc580808750f73ca', 'c0be585ed21b1a6c6dc9559ebe007ede', 'fa5292c6c728810f6bfee095a3824dbd', 'f0066146566567f146da71ff8ec650c9', '983a5160b1ba3ff7dbde7e636e4c4601']


In [21]:
import pandas
import surprise
from surprise import Dataset, Reader
from surprise.model_selection import cross_validate, GridSearchCV
from tables.tests import test_suite
from collections import defaultdict

import numpy

## Loading the dataset into memory

In [40]:
#import the data
raw_core = pandas.read_csv('fashion.csv')

#cleaning the data of whitespace
raw_core["ProductID"] = raw_core["ProductID"].str.strip()
raw_core["SceneID"] = raw_core["SceneID"].str.strip()

#removing the unused column
raw_shirts = pandas.read_csv("shirts.csv").drop('Scene',axis=1)
raw_bottoms = pandas.read_csv("bottoms.csv").drop('Scene',axis=1)
raw_shoes = pandas.read_csv("shoes-only.csv").drop('Scene',axis=1)

In [41]:
#adding dummy rating for sorting
raw_core["Rating"] = 1

## Adding bandits
need to add random zero values to prevent the matrix from just zeroing out.

In [42]:
# query
obfuscation_percentage = 0.10
obfuscation_cycles = 1

# need to define a function that will add 
def obfuscate(sample_frame, percentage, cycles):
  for i in range(0, cycles):
    #np arrays
    unique_products = sample_frame["ProductID"].unique()
    unique_scenes = raw_core["SceneID"].unique()

    #panda dataframes
    df_uproducts = pandas.DataFrame(unique_products)
    df_uscenes = pandas.DataFrame(unique_scenes)

    #getting only a subset of the dataframes to work with
    product_subset = df_uproducts.sample(frac = obfuscation_percentage)

    #print(product_subset)
    for item in product_subset[0]:
      #getting a random scene as a DF
      random_scene_df = df_uscenes.sample()

      # converting to string
      random_scene = random_scene_df.iloc[0][0]

      #Getting the list from the dictionary to check against
      buffer_scenes = scene_list[item]

      #Check if item is there, if add it
      if random_scene in buffer_scenes:
        #this case is only if the item exist as a pair already
        print("Collision PID:" + item + ": SID:" + random_scene + ":")

      else:
        #print("before update " + str(scene_list[item]))
        #add scene to dictionary
        scene_list[item].append(random_scene.strip())

        #print("after update " + str(scene_list[item]))
        #add scene to dataframe as zero
        raw_core.loc[len(raw_core.index)] = [item, random_scene, 0]

In [43]:
clothing_pd_array = [raw_shirts, raw_bottoms, raw_shoes]

for array in clothing_pd_array:
  obfuscate(array, obfuscation_percentage, obfuscation_cycles)

Collision PID:88d76bd43ccf1d26d2f29c4c91396bee: SID:cee45144edcdedd83918e119cf01b811:


##Testing code do NOT run

In [27]:
#sratch for testing delete afterwards!
obfuscation_percentage = 10

#create a list of unique items
## this will become a numpys array
unique_products = raw_shirts["ProductID"].unique()
unique_scenes = raw_core["SceneID"].unique()

df_uproducts = pandas.DataFrame(unique_products)
df_uscenes = pandas.DataFrame(unique_scenes)

#getting a sublist of products
item_df = df_uproducts.sample()


random_scene_df = df_uscenes.sample()
#should have cdab9160072dd1800038227960ff6467 14f59334af4539132981b1324a731067 
# e7d32df9f45b691afc580808750f73ca c0be585ed21b1a6c6dc9559ebe007ede fa5292c6c728810f6bfee095a3824dbd f0066146566567f146da71ff8ec650c9 983a5160b1ba3ff7dbde7e636e4c4601 

# need to use to convert 
item = item_df.iloc[0][0]
random_scene = random_scene_df.iloc[0][0]

#item to test (TESTING ONLY)
item = "0027e30879ce3d87f82f699f148bff7e"
random_scene = "e7d32df9f45b691afc580008750f73ca"

print("productID =" + str(item) + ": Scene =" + str(random_scene)+":")

productID =0027e30879ce3d87f82f699f148bff7e: Scene =e7d32df9f45b691afc580008750f73ca:


In [29]:
buffer_scenes = scene_list[item]
print(buffer_scenes)

print("Current DataFrame is")
print(raw_core["ProductID"].str.contains(item, na = False))
print("\n")

if random_scene in buffer_scenes:
  # get a new scene
  print("buffer found")
else:
  print("before update " + str(scene_list[item]))
  #add scene to dictionary
  scene_list[item].append(random_scene.strip())

  print("after update " + str(scene_list[item]))
  #add scene to dataframe as zero
  raw_core.loc[len(raw_core.index)] = [item, random_scene, 0]

print("After DataFrame is")
print(raw_core["ProductID"].str.contains(item, na = False))
print("\n")


['cdab9160072dd1800038227960ff6467', '14f59334af4539132981b1324a731067', 'e7d32df9f45b691afc580808750f73ca', 'c0be585ed21b1a6c6dc9559ebe007ede', 'fa5292c6c728810f6bfee095a3824dbd', 'f0066146566567f146da71ff8ec650c9', '983a5160b1ba3ff7dbde7e636e4c4601', 'e7d32df9f45b691afc580008750f73ca']
Current DataFrame is
0         True
1         True
2         True
3         True
4        False
         ...  
72194    False
72195    False
72196    False
72197    False
72198     True
Name: ProductID, Length: 72199, dtype: bool


buffer found
After DataFrame is
0         True
1         True
2         True
3         True
4        False
         ...  
72194    False
72195    False
72196    False
72197    False
72198     True
Name: ProductID, Length: 72199, dtype: bool




## Loading the subsets into memory

In [44]:
shirts_only = raw_core[raw_core['ProductID'].isin(raw_shirts['ProductID'])]
bottoms_only = raw_core[raw_core['ProductID'].isin(raw_bottoms['ProductID'])]
shoes_only = raw_core[raw_core['ProductID'].isin(raw_shoes['ProductID'])]

reader = Reader(rating_scale=(0,1))

full_shirts = Dataset.load_from_df(shirts_only,reader)
full_bottoms = Dataset.load_from_df(bottoms_only,reader)
full_shoes = Dataset.load_from_df(shoes_only,reader)

## Hyper parameter searches, prototyping do not using.

In [ ]:
#Param for grid search
param_grid ={'n_factors': [2,5,10,20,50,100],
             'reg_all': [0.02,0.05,0.1],
             'n_epochs': [10,20,50]}

reader = Reader(rating_scale=(0,1))

In [ ]:
#note The dataframe containing the ratings. It must have three columns, corresponding to the user (raw) ids, the item (raw) ids, and the ratings, in this order.
shirt_sample = shirts_only.sample(frac=0.1)

full_shirt = Dataset.load_from_df(shirts_only,reader)
sample_shirt = Dataset.load_from_df(shirt_sample,reader)

In [ ]:
gs = GridSearchCV(surprise.SVD, param_grid,measures=['rmse','mae'],cv=5,n_jobs=-1, joblib_verbose=2)

In [ ]:
gs.fit(sample_shirt)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  37 tasks      | elapsed:    3.0s
[Parallel(n_jobs=-1)]: Done 267 out of 270 | elapsed:   21.3s remaining:    0.2s
[Parallel(n_jobs=-1)]: Done 270 out of 270 | elapsed:   21.4s finished


In [ ]:
print(gs.best_score['rmse'])
print(gs.best_params['rmse'])
print(gs.best_params['rmse'])

0.0009149596462545342
{'n_factors': 2, 'reg_all': 0.02, 'n_epochs': 10}
{'n_factors': 2, 'reg_all': 0.02, 'n_epochs': 10}


## HyperParameter Tuner Functions

In [ ]:
#defaults parameters
hyperDefaults = {'n_factors': 2, 'reg_all': 0.02, 'n_epochs': 10}

In [ ]:
#Param for grid search
param_grid ={'n_factors': [2,5,10,20,50,100],
             'reg_all': [0.02,0.05,0.1],
             'n_epochs': [10,20,50]}

In [ ]:
def hyperSearcher(clothing_dataset):
  clothing_sample_ds = clothing_datset.sample(frac=0.1)

  clothing_sample = sample_shirt = Dataset.load_from_df(clothing_sample_ds,reader)

  gs = GridSearchCV(surprise.SVD, param_grid,measures=['rmse','mae'],cv=5,n_jobs=-1, joblib_verbose=2)
  gs.fit(clothing_sample)

  print(gs.best_score['rmse'])
  print(gs.best_params['rmse'])

  return gs.best_params['rmse']

## Training and Predicting the models


In [45]:
#training the shirt model
shirts_model = surprise.SVD(n_factors=2,reg_all=0.05, n_epochs=10)
shirts_suprirse_data = full_shirts.build_full_trainset()
shirts_model.fit(shirts_suprirse_data)

bottoms_model = surprise.SVD(n_factors=2,reg_all=0.05, n_epochs=10)
bottoms_suprirse_data = full_bottoms.build_full_trainset()
bottoms_model.fit(bottoms_suprirse_data)

shoes_model = surprise.SVD(n_factors=2,reg_all=0.05, n_epochs=10)
shoes_suprirse_data = full_shoes.build_full_trainset()
shoes_model.fit(shoes_suprirse_data)

This function is to return the first x items that have a perfect hit for matches.



In [58]:
def first_hit_list(algo, length, ):

  #prereq getting a unique productIDs
  unique_products = raw_core["ProductID"].unique()

  #getting the source item list of scenes
  list_of_scenes = master_scene_list[source_item]

  #buffer to hold the output values
  output_dataframe = pandas.DataFrame(columns = ['ProductID', 'r_value'])

  for scene in list_of_scenes:
    for product in unique_products:
      if product != source_item:
        output = algo.predict(product, scene)

        output_dataframe = output_dataframe.append({'ProductID' : output[0], 'r_value': output[3]}, ignore_index = True)

  output_dataframe = output_dataframe.sort_values(by='r_value', ascending=False)
  top_ten_df = output_dataframe.head(10)

  return top_ten_df['ProductID'].tolist()

In [59]:
shirt_top_ten = first_hit_list(shirts_model, 10)
bottom_top_ten = first_hit_list(bottoms_model,10)
shoes_top_ten = first_hit_list(shoes_model,10)

## Results

In [60]:
print("Recommended shirts" + str(shirt_top_ten))
print("Recommended Bottoms" + str(bottom_top_ten))
print("Recommended Shotes" + str(shoes_top_ten))

Recommended shirts['6d7c884b123c3bc73159236efa5e0fd6', '81670d6c00b6a43f05c287bbde5aa867', '8c9342e5ccd72353dd607d80cb78e63f', 'e251be60024247981999ba5a38b53894', 'e7c43541470d3701877004d5b4f61f1f', 'c39ddd5567e58aba0538b6008c71d614', '86ac6af55fa93bd0212d431af8c2a8c1', 'a59ccc8a80a07f20e88ebfbc0f613b16', 'cdff384f701a023e3d8824d1d7c44e5f', '42ded1188980f834f969c16f166ae561']
Recommended Bottoms['24c393d19ea2711e7325596b9f131efa', '2be0e67f129c0699a109b2a9b1f25465', '6f7ca682f2bc192915c5d823ac4aa2bb', '4882e8f87d2c69c8473dc5230e6fb631', 'f5fa453c791f4836e6b6bb602ab03b1c', '115fc198be0000834f95aeabdd1dc989', '8b5832a1c4ed3ad342ab572693123be2', '63dcaf508898edc7316fa46ed9107765', '5faeb2038483b83e989a57b8c6c01198', 'c32ba4fe9f4854eca7310131c78bfaf6']
Recommended Shotes['f4f0b457ddc25adbe25a5f0cc659536f', '821b7a91ed609ddace7917daed2cc199', 'ae13233b0ed409149239a49798b292f4', '7ff253d3cb0ea118d604b995fe01edbf', 'e2248d94efadd04132a4ee66a9f32c4c', '262be35a82e6af56a95d68cb388485ee', '1e93a